<a href="https://colab.research.google.com/github/EllaThomasson/EDA/blob/main/Wrangling_Solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Question 1

1. The author of this paper emphasizes the importance of having clean data. This paper generally seems to be about making a clear definition of data cleaning. Wickham data is clean when each row has exactly one observation, when each column houses exactly one variable, and when each type of observational unit is a table.

2. The tidy data standard was designed to facilitate initial exploration and analysis of the data. It is meant to make the data cleaning process easier by helping tools work well together. This allows you to focus on more interesting domain problems.

3. The first quote means that organized or well-structured datasets share common characteristics, like being clean, complete, and easy to analyze. However, messy datasets each have their own unique issues, like missing values, inconsistencies, or noise—making them distinct in their chaos. The second sentence suggests that while it's often straightforward to identify the elements of a specific dataset—such as which items are observations and which are variables—it can be challenging to provide a clear and universal definition of what constitutes observations and variables across different contexts or datasets.

4. A dataset is a collection of values. A variable measures the same attribute of all observations. An observation contains all values measured on the same unit.

5. Tidy data is where each variable forms a column, each observation forms a row, and each type of observational unit forms a table.

6. Some common problems with messy data are: 1. Column headers are values and not names. In this case, variables form both the rows and the columns, and column headers are values, not variable names. This forms a problem. 2. There are multiple variables stored in one column. This often happens after melting, where the column variable names often become a combination of multiple underlying variable names. 3. Variables are stored in both rows and columns. This is the most complex form of messy data. 4. Multiple types in one table. During tidying, each type of observational unit should be stored in its own table. 5. One type in multiple tables. Generally a pretty easy problem to fix. In table 4, income should be a variable, not shown as values across the columns. In this case, the column headers are values and not names. Melting is the process of fixing this issue. It is the process of converting column-value variables into rows.

7. For starters, table 11 has an issue with values being in the columns. More specifically, the days are in the columns, which need to be brought down into a row. This is fixed in table 12, and a date variable was made combining month, year, and day. Table 12 made the values into one column as well, which is good. Overall, table 12 is good because each column is a variable and each row is an observation. This was not the case with table 11.

8. The chicken and egg problem with focusing on tidy data is that if tidy data is only as useful as the tools that work with it, then tidy tools will always be linked to tidy data with no way out. Because of this, it is easy to get stuck in a local maxima where independently changing data structures/tools will not improve workflow. Breaking out of this local maxima will take long-term consistent effort. In the future, Wickham hopes that the tidy framework isn't a false start and that people continue to grow it and build upon it. Additionally, Wickham wants the tidy concept to be bigger than just technical concepts. He wants it to be more about the bigger ideas and tools for data science more generally.



In [9]:
import pandas as pd
import numpy as np

Question 2

In [3]:
! git clone https://github.com/DS3001/wrangling

Cloning into 'wrangling'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 92 (delta 36), reused 17 (delta 14), pack-reused 43 (from 1)
Receiving objects: 100% (92/92), 18.19 MiB | 6.97 MiB/s, done.
Resolving deltas: 100% (41/41), done.


Part 1

In [4]:
airbnb = pd.read_csv('./wrangling/assignment/data/airbnb_hw.csv', low_memory=False)

# first took a look at the data type / values

airbnb['Price'].value_counts().sort_values()

(airbnb['Price']).unique() # here I can see that the observations are recorded as strings because there is a comma in one + numbers

# so I need to take out the commas and change the column to numeric

airbnb['Price'] = airbnb['Price'].str.replace(',','')

pd.to_numeric(airbnb['Price'])


# now I'm taking a look at missing values

airbnb['Price'].isnull().sum() # there are no missing values, so we are all good here


0

Part 2

In [26]:

sharks = pd.read_csv('./wrangling/assignment/data/sharks.csv', low_memory=False)

# again, first I'm taking a look at all the values

sharks['Type'].value_counts()

## I see that some values are the same\similar, so I will combine them

sharks['Type'] = sharks['Type'].replace('Boating', 'Boat')
sharks['Type'] = sharks['Type'].replace('Boatomg', 'Boat')
sharks['Type'] = sharks['Type'].replace('Sea Disaster', 'Boat')
sharks['Type'] = sharks['Type'].replace('Watercraft', 'Boat')
sharks['Type'].value_counts()

## all of the following variables are unknown for some reason, so I will make them all NA

sharks['Type'] = sharks['Type'].replace(['Invalid', 'Questionable','Unconfirmed','Unverified','Under investigation'],np.nan)
sharks['Type'].value_counts()


## I will not remove the NAs yet, because it may be useful to know when the "type" was questionable.


Part 3

In [11]:

justice = pd.read_parquet('./wrangling/assignment/data/justice_data.parquet')

# taking a look at the data first

justice['WhetherDefendantWasReleasedPretrial'].unique() # 9 means "unclear" ... so lets change that to NAN

justice['WhetherDefendantWasReleasedPretrial'] = justice['WhetherDefendantWasReleasedPretrial'].replace(9,np.nan)

# double checking to make sure we fixed the problem

justice['WhetherDefendantWasReleasedPretrial'].value_counts() # this looks good!


,count
WhetherDefendantWasReleasedPretrial,
1.0,19154
0.0,3801


Part 4

In [5]:

pretrial = pd.read_csv('./wrangling/pretrial_data.csv', low_memory=False)

pretrial['ImposedSentenceAllChargeInContactEvent'] = pd.to_numeric(pretrial['ImposedSentenceAllChargeInContactEvent'],errors='coerce')

pretrial['ImposedSentenceAllChargeInContactEvent'].isnull().sum() / 22986
# almost 40% NA values

subset = pretrial[['ImposedSentenceAllChargeInContactEvent','SentenceTypeAllChargesAtConvictionInContactEvent']]


# after looking at the data, it appears as though when the SentenceTypeAllChargesAtConvictionInContactEvent
# is 4 and 9, the ImposedSentenceAllChargeInContactEvent is more likely to be missing


# Category 4 is cases where the charges were dismissed therefore this value should be 0 not NA

pretrial['ImposedSentenceAllChargeInContactEvent'] = pretrial['ImposedSentenceAllChargeInContactEvent'].mask( pretrial['SentenceTypeAllChargesAtConvictionInContactEvent'] == 4, 0)


subset = pretrial[['ImposedSentenceAllChargeInContactEvent','SentenceTypeAllChargesAtConvictionInContactEvent']]


# 9s should actually be NA

subset['ImposedSentenceAllChargeInContactEvent'].isnull().sum() / 22986

# now there are only 274 missing values ... which is only around 1% of the data, much better



0.011920299312625076